In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


def setup_gpu():
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)

#     policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy('mixed_float16')        

setup_gpu()

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


In [2]:
sns.set(rc={'figure.figsize':(16,8)})

In [3]:
(raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = tf.keras.datasets.imdb.load_data()
word_index = tf.keras.datasets.imdb.get_word_index()

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/aki/Killara/Training_Data_On_Imbalanced_Text_Data/venv/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/aki/Killara/Training_Data_On_Imbalanced_Text_Data/venv/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple 

In [4]:
MAX_WORDS = 256
padded_x_train = tf.keras.preprocessing.sequence.pad_sequences(raw_x_train, maxlen=MAX_WORDS)
padded_x_test = tf.keras.preprocessing.sequence.pad_sequences(raw_x_test, maxlen=MAX_WORDS)


In [8]:
# positive_idx = np.where(raw_y_train > 0)[0]
# negative_idx = np.where(raw_y_train == 0)[0]

# np.random.shuffle(positive_idx)
# positive_last = int(len(positive_idx) * 0.5)
# positive_idx = positive_idx[:positive_last]

# positive_idx = np.tile(positive_idx, 2)

# idx = np.concatenate((positive_idx, negative_idx))

# x_train = padded_x_train[idx]
# y_train = raw_y_train[idx]

x_train = padded_x_train
y_train = raw_y_train

x_test = padded_x_test
y_test = raw_y_test

def vectorize(sequences, longest = MAX_WORDS):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [9]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

print(f"word_index size: {len(word_index)}")

x_train shape: (25000, 256)
y_train shape: (25000,)
x_test shape: (25000, 256)
y_test shape: (25000,)
word_index size: 88584


In [10]:
train_positive = np.where(y_train > 0)[0]
train_negative = np.where(y_train == 0)[0]

In [11]:
print(f"Positive reviews: {train_positive.shape}")
print(f"Negative reviews: {train_negative.shape}")

Positive reviews: (12500,)
Negative reviews: (12500,)


In [14]:
vsz = len(word_index) - 1

EMBEDDING_SIZE = 16
HIDDEN_LAYER_SIZE = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vsz, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          5669312   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               197632    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,883,457
Trainable params: 5,883,457
Non-trainable params: 0
_________________________________________________________________


In [15]:
opt = tf.keras.optimizers.Adam()
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2000, batch_size=128, verbose=2)



Epoch 1/2000


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[32,199] = 88584 is not in [0, 88583)
	 [[node sequential_1/embedding_1/embedding_lookup (defined at <ipython-input-15-4ebfc5fbf214>:4) ]]
  (1) Invalid argument:  indices[32,199] = 88584 is not in [0, 88583)
	 [[node sequential_1/embedding_1/embedding_lookup (defined at <ipython-input-15-4ebfc5fbf214>:4) ]]
	 [[ConstantFoldingCtrl/cond/switch_pred/_2_0/_35]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_12655]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
 sequential_1/embedding_1/embedding_lookup/10151 (defined at /usr/lib/python3.8/contextlib.py:113)

Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
 sequential_1/embedding_1/embedding_lookup/10151 (defined at /usr/lib/python3.8/contextlib.py:113)

Function call stack:
train_function -> train_func

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))